In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from fancyimpute import KNN
from sklearn.impute import SimpleImputer

Using TensorFlow backend.


In [43]:
df = pd.read_excel('brand_characteristics_data_file.xlsx')
print(df.shape)


(697, 73)


# Dropping the nan rows 

In [45]:
#df1 = df.dropna()
#print("Shape of the dataset after dropping all the rows with missing data: ", np.shape(df))

df2 = df.dropna(subset = ['Brand_Stature_C'])
print("Shape of the dataset after dropping all the rows with missing data: ", np.shape(df2))
missing_val = df2.Brand_Asset_C.isnull().sum()  # Checking if any of the 3 target variables have nan values
print(missing_val) 
#df[df.Brand_Stature_C != 'Nan']

Shape of the dataset after dropping all the rows with missing data:  (697, 73)
Shape of the dataset after dropping all the rows with missing data:  (629, 73)
0


In [46]:
# I have used df2 as input to the model - because I did not want to have missing data in the y_train too !!
# df2 is obtained after dropping rows with nan values in columns Brand_stature_C

useless_cols = ['Category', 'Brand_id', 'Brand_name']
data = df2.drop(useless_cols, axis=1)
data.loc[:, 'Premium']
data.Premium.replace(('Value', 'Middle', 'Premium'), (1, 2, 3), inplace=True) # 3 - Value == Budget brand
onehot_v1 = pd.get_dummies(data.loc[:, 'Premium'], prefix='Premium')
data = data.join(onehot_v1)
data = data.drop('Premium', axis=1)

onehot_v1 = pd.get_dummies(data.loc[:, 'Category_code'], prefix='Category_code')
data = data.join(onehot_v1)
data = data.drop('Category_code', axis=1)

In [47]:
feature_list = "Category_code, Age, Newness_to_category, \
Total_Users_pct, Total_Prefer_pct, Energized_Differentiation_C, Relevance_C, Esteem_C, \
Knowledge_C, Different_pct, Distinctive_pct, \
Unique_pct, Dynamic_pct, Innovative_pct, Leader_pct, Reliable_pct, High_quality_pct, \
Arrogant_pct, Authentic_pct, Best_Brand_pct, Carefree_pct, Cares_Customers_pct, Charming_pct, \
Daring_pct, Down_to_Earth_pct, Energetic_pct, Friendly_pct, Fun_pct, Gaining_In_Popularity_pct, \
Glamorous_pct, Good_Value_pct, Healthy_pct, Helpful_pct, High_Performance_pct, Independent_pct, \
Intelligent_pct, Kind_pct, Obliging_pct, Original_pct, Prestigious_pct, Progressive_pct, \
Restrained_pct, Rugged_pct, Sensuous_pct, Simple_pct, Social_pct, Socially_Responsible_pct, \
Straightforward_pct, Stylish_pct, Traditional_pct, Trendy_pct, Trustworthy_pct, \
Unapproachable_pct, Up_To_Date_pct, Upper_Class_pct, Visionary_pct, Worth_More_pct, \
Cutting_Edge_C, Classic_C, Superior_C, Chic_C, Customer_Centric_C, Outgoing_C, No_Nonsense_C, \
Distant_C, Regard_MS"

feature_list = feature_list.split(', ')
#print imp_features
print(len(feature_list))


66


In [48]:
train = data[:450]
val = data[450:550]
test = data[550:]
print("Train dataset: ", len(train))
print("Test dataset: ", len(test))
print("Val dataset: ", len(val))

y_labels = ['Brand_Stature_C', 'Brand_Strength_C', 'Brand_Asset_C']

y_train = train[y_labels]
X_train = train.drop(y_labels, axis=1)


y_val = val[y_labels]
X_val = val.drop(y_labels, axis=1)

y_test = test[y_labels]
X_test = test.drop(y_labels, axis=1)

mean_Xtrain = np.mean(X_train)
display(mean_Xtrain)

Train dataset:  450
Test dataset:  79
Val dataset:  100


Age                            58.811160
Newness_to_category            65.439750
Total_Users_pct                33.939015
Total_Prefer_pct               40.930202
Energized_Differentiation_C     0.494656
Relevance_C                     2.752115
Esteem_C                        0.622283
Knowledge_C                     3.579079
Different_pct                   7.449110
Distinctive_pct                10.459243
Unique_pct                      7.748723
Dynamic_pct                     7.604813
Innovative_pct                  7.959433
Leader_pct                     15.942557
Reliable_pct                   17.516719
High_quality_pct               18.048109
Arrogant_pct                    4.902784
Authentic_pct                   9.168356
Best_Brand_pct                  9.052178
Carefree_pct                    5.920752
Cares_Customers_pct            11.567063
Charming_pct                    6.076843
Daring_pct                      6.603166
Down_to_Earth_pct              19.176997
Energetic_pct   

In [49]:
from sklearn.preprocessing import Imputer
impute_func = SimpleImputer(missing_values=np.nan, strategy='mean')
for i, feature in enumerate(feature_list[2:]):
    #print(i)
    X_train[feature] = impute_func.fit_transform(X_train[feature].values.reshape(-1, 1))

In [50]:
#impute_func.fit_transform(train.loc[:,feature_list[7]].values.reshape(-1,1))

X_train[feature_list[10]]

1      12.199670
3      13.322060
4      13.917360
5       6.879766
6      14.854470
7      10.308100
8       8.055678
9       9.503695
10     10.302430
11      9.351686
12      8.212419
13      6.346408
14      6.817083
16      7.171248
17      2.919549
18      8.007871
19      6.785540
20      7.955283
21      3.986943
22      9.663995
23      9.740601
24     11.556740
26     11.609190
27      8.418119
28      8.714547
29     10.120280
30     13.704010
32      9.869622
33      6.565214
34     11.491470
         ...    
467    10.560410
468    13.628270
469    12.065790
471    17.603451
472    14.018850
473    12.995370
474     9.317093
475    11.339310
476    11.420750
477    10.046480
478    11.752410
479    10.472030
480    13.609890
481     6.405635
482     9.073104
483     6.148084
484    11.380370
486    10.487360
487     9.682291
488    11.687460
490    19.295490
491     3.883129
492    14.075120
493    12.555390
494     6.227262
495    12.544120
496    10.358460
497     7.8903

In [51]:
category_list = "Beauty products, Beverages, Cars, Children's products, Clothing products, Department stores, Financial services, Food and dining, Health products and services, Home design and decoration, Household Products, Media and entertainment, Sports and hobbies, Technology products and stores, Telecommunications, Travel services"
category_list = category_list.split(', ')
print(category_list)

a = dict(Counter(df.Category))
a

['Beauty products', 'Beverages', 'Cars', "Children's products", 'Clothing products', 'Department stores', 'Financial services', 'Food and dining', 'Health products and services', 'Home design and decoration', 'Household Products', 'Media and entertainment', 'Sports and hobbies', 'Technology products and stores', 'Telecommunications', 'Travel services']


{'Beauty products': 52,
 'Beverages': 66,
 'Cars': 47,
 "Children's products": 19,
 'Clothing products': 51,
 'Department Stores': 15,
 'Financial services': 39,
 'Food and dining': 105,
 'Health products and services': 27,
 'Home design and decoration': 13,
 'Household Products': 24,
 'Media and entertainment': 103,
 'Sports and hobbies': 21,
 'Technology products and stores': 56,
 'Telecommunications': 25,
 'Travel services': 34}

In [52]:
# Checking for Nan values in X_train and summing them up
#print(x_train.isnull().any())
missing_val = X_train.isnull().sum()
print(missing_val.sum()) 

# Prints a non zero value if there are any nan's

0


In [53]:
# Print the missing data in all the columns - Errors due to category_code one hot encoding !!!!!!!!!!!


# print("Printing the number of missing values in each feature...")
# for counter, feature in enumerate(feature_list):
#     print(counter, feature)
#     column = x_train[feature]
#     print((column.isnull()==True).sum())

In [54]:
# Checking if the imputation has been done properly - since there are a lot of missing values, 
# - mode(most repeated value) should be the mean value
from scipy import stats
stats.mode(X_train[feature_list[63]])


ModeResult(mode=array([3.33250189]), count=array([1]))

In [55]:
#Performing PCA

#Standardizing the values before PCA
from sklearn.preprocessing import StandardScaler
Xtrain_std = StandardScaler().fit_transform(X_train)
display(Xtrain_std)

/home/ak/env/lib/python3.5/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ak/env/lib/python3.5/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[-1.25272078, -0.16124743, -0.26628422, ..., -0.28131244,
        -0.16552118, -0.23735633],
       [ 0.552325  , -0.10185494,  1.23846537, ..., -0.28131244,
        -0.16552118, -0.23735633],
       [ 0.80977742, -0.25906298,  1.05293888, ..., -0.28131244,
        -0.16552118, -0.23735633],
       ...,
       [-0.83089912, -0.95385888,  0.34013078, ..., -0.28131244,
        -0.16552118, -0.23735633],
       [-1.03125413, -0.09775313, -0.60400836, ..., -0.28131244,
        -0.16552118, -0.23735633],
       [ 2.57314826, -0.60252654,  0.        , ..., -0.28131244,
        -0.16552118, -0.23735633]])

In [56]:
# Feature selection using PCA
from sklearn.decomposition import PCA
pca = PCA()
principalComponents = pca.fit_transform(X_train)

print("Number of components before PCA: ", np.shape(X_train))
print("Number of components after PCA: ", np.shape(principalComponents))

#principalDf = pd.DataFrame(data = principalComponents
#             , columns = ['principal component 1', 'principal component 2'])

# Finding the most important features by choosing the features which have the highest variance

variance_ratio = pca.explained_variance_ratio_
variance_ratio.sort(axis=0)
variance_ratio = variance_ratio[::-1]

# Taking the first n features which cause the variance to sum up to 0.9
sum=0
for i, item in enumerate(variance_ratio):
    if(sum>=0.9):
        break
    sum+=item
print("N values which sum upto 0.9: ", i)
principal_list = variance_ratio[:i]

# Picking out the top n components 
final_components = []
for item in principal_list:
    column_idx = variance_ratio.tolist().index(item)  # the variance_ratio is a numpy array array, 
                            # hence converting to a list and finding the indexes of the columns to be picked out 
    final_components.append(principalComponents[:,column_idx])
X_train_final = np.transpose(np.array(final_components))
print("Final size of the X_train after PCA: ", np.shape(X_train_final))

Number of components before PCA:  (450, 85)
Number of components after PCA:  (450, 85)
N values which sum upto 0.9:  5
Final size of the X_train after PCA:  (450, 5)


In [59]:
# Fitting a model 
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, max_depth=max_depth))
regr_multirf.fit(X_train, y_train)

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,random_state=2)
regr_rf.fit(X_train, y_train)

# Predict on new data
y_multirf = regr_multirf.predict(X_test)
y_rf = regr_rf.predict(X_test)

# Calculating the accuracy of prediction accuracy
score_multirf = regr_multirf.score(X_test, y_test)
print("Multi RF score: ", score_multirf)

score_rf = regr_rf.score(X_test, y_test)
print("RF score: ", score_rf)


Multi RF score:  0.9576358397996646
RF score:  0.9457278805071456


In [25]:
students = [ ('jack', 34, 'Sydeny') ,
             ('Riti', float('nan'), 'Delhi' ) ,
             ('Aadi', 16, float('nan')) ]

# Create a DataFrame object
dfObj = pd.DataFrame(students, columns = ['Name' , 'Age', 'City'], index=['a', 'b', 'c'])
dfObj

dfObj = dfObj.dropna()


dfObj.loc[:,'Age']

a    34.0
Name: Age, dtype: float64

# Observations
Number of missing rows : 79 (For total_users_pct and Total_Prefer_pct)
                         68 (For other percentages)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor


# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(600, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y += (0.5 - rng.rand(*y.shape))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=400, test_size=200, random_state=4)

print(np.shape(y_train))
max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
                                                          max_depth=max_depth,
                                                          random_state=0))
regr_multirf.fit(X_train, y_train)

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
                                random_state=2)
regr_rf.fit(X_train, y_train)

# Predict on new data
y_multirf = regr_multirf.predict(X_test)
y_rf = regr_rf.predict(X_test)

# Plot the results
plt.figure()
s = 50
a = 0.4
plt.scatter(y_test[:, 0], y_test[:, 1], edgecolor='k',
            c="navy", s=s, marker="s", alpha=a, label="Data")
plt.scatter(y_multirf[:, 0], y_multirf[:, 1], edgecolor='k',
            c="cornflowerblue", s=s, alpha=a,
            label="Multi RF score=%.2f" % regr_multirf.score(X_test, y_test))
plt.scatter(y_rf[:, 0], y_rf[:, 1], edgecolor='k',
            c="c", s=s, marker="^", alpha=a,
            label="RF score=%.2f" % regr_rf.score(X_test, y_test))
plt.xlim([-6, 6])
plt.ylim([-6, 6])
plt.xlabel("target 1")
plt.ylabel("target 2")
plt.title("Comparing random forests and the multi-output meta estimator")
plt.legend()
plt.show(2)

In [ ]:
!pip install matplotlib
